# Alternative Preprocessing Strategy for Classification

This notebook implements a robust preprocessing strategy for classification, with particular attention to preventing data leakage and ensuring reproducibility.

## Key Principles
- The train/test split is performed before any transformation.
- STKDE parameters are optimized only on the training data.
- All feature/label engineering is postponed to the modeling phase.
- Preprocessing pipelines are defined here but fitted only on the training data during modeling.

**Note:**
- All custom functions (e.g., cyclical_transform, BinarizeSinCosTransformer) are defined in `custom_transformers.py` to ensure modularity and reusability.
- The produced artifacts (data, pipeline, parameters, scoring_dict) are used by `Modeling.ipynb`.

# Setup

Import libraries, define paths, and prepare for preprocessing. All custom functions are imported from `custom_transformers.py` where needed.

## Google Drive Mount (optional)

If working locally, this cell can be ignored.

In [1]:
# from google.colab import drive
# drive.mount('/drive', force_remount=True)

### Import libraries

In [2]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import json
from typing import List, Dict, Any, Tuple, Union
import time
from sklearn.model_selection import train_test_split
from sklearn.neighbors import BallTree
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer, StandardScaler, KBinsDiscretizer, Binarizer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import random
import joblib
from joblib import Parallel, delayed
from sklearn.neighbors import KDTree
from pyproj import Transformer
from Utilities.custom_transformers import cyclical_transform

random.seed(42)
np.random.seed(42)

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore', category=FutureWarning)
print("Libraries imported and random seeds set.")


Libraries imported and random seeds set.


## Path Definition

Define paths for loading data and saving preprocessing artifacts.

In [3]:
import os

base_dir = r"C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs"
feature_engineered_file_path = os.path.join(base_dir, "Final", "final_crime_data.csv")
save_dir = os.path.join(base_dir, "Classification (Preprocessing)")
os.makedirs(save_dir, exist_ok=True)

print(f"Notebook directory: {base_dir}")
print(f"Feature engineered file path: {feature_engineered_file_path}")
print(f"Save directory: {save_dir}")

Notebook directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs
Feature engineered file path: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Final\final_crime_data.csv
Save directory: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Classification (Preprocessing)


### Load and validate feature engineered data

Load the dataset produced by the initial feature engineering phase.

In [4]:
import pandas as pd

def load_basic_data_info(file_path: str) -> pd.DataFrame:
    df = pd.read_csv(file_path)
    print(f"Loaded data: {df.shape[0]} rows, {df.shape[1]} columns")
    print("Columns:", df.columns.tolist())
    print(df.info())
    return df

df = load_basic_data_info(feature_engineered_file_path)

Loaded data: 2493835 rows, 44 columns
Columns: ['BORO_NM', 'KY_CD', 'LAW_CAT_CD', 'LOC_OF_OCCUR_DESC', 'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON', 'TIME_BUCKET', 'IS_HOLIDAY', 'IS_PAYDAY', 'SAME_AGE_GROUP', 'SAME_SEX', 'TO_CHECK_CITIZENS']
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2493835 entries, 0 to 2493834
Data columns (total 44 columns):
 #   Column              Dtype  
---  ------              -----  
 0   BORO_NM             object 
 1   KY_CD               int64  
 2   LAW_CAT_CD          object 
 3   LOC_OF_OCCUR_DESC 

## Data cleaning

- Remove irrelevant rows (`TO_CHECK_CITIZENS` = 0)
- Remove rows where `VIC_SEX` is not in ['M', 'F']
- Remove rows with unknown location
- Remove columns not available at prediction time

These steps ensure consistency with operational conditions and reliability of performance metrics.

In [5]:
print("=== Data Cleaning ===")
initial_rows = len(df)
df_cleaned = df.copy()

# Remove rows not relevant to citizens
if 'TO_CHECK_CITIZENS' in df_cleaned.columns:
    citizen_mask = df_cleaned['TO_CHECK_CITIZENS'] != 0
    rows_removed = (~citizen_mask).sum()
    df_cleaned = df_cleaned[citizen_mask]
    print(f"Removed {rows_removed} rows where TO_CHECK_CITIZENS = 0") 

# Remove rows where VIC_SEX is not in ['M', 'F']
if 'VIC_SEX' in df_cleaned.columns:
    valid_sex_mask = df_cleaned['VIC_SEX'].isin(['M', 'F'])
    rows_removed = (~valid_sex_mask).sum()
    df_cleaned = df_cleaned[valid_sex_mask]
    print(f"Removed {rows_removed} rows where VIC_SEX is not 'M' or 'F'. Rows remaining: {len(df_cleaned)}")
else:
    print("'VIC_SEX' column not found, skipping VIC_SEX filter.")

# Remove rows where the location is unknown
if 'LOC_OF_OCCUR_DESC' in df_cleaned.columns:
    if df_cleaned['LOC_OF_OCCUR_DESC'].dtype == 'object':
        location_mask = df_cleaned['LOC_OF_OCCUR_DESC'].str.upper() != 'UNKNOWN'
        rows_removed = (~location_mask).sum()
        df_cleaned = df_cleaned[location_mask]
        print(f"Removed {rows_removed} rows with UNKNOWN LOC_OF_OCCUR_DESC. Rows remaining: {len(df_cleaned)}") 
    else:
        print("'LOC_OF_OCCUR_DESC' is not of object type, skipping UNKNOWN location filter. It might be OHE.")
else:
    print("'LOC_OF_OCCUR_DESC' column not found, skipping UNKNOWN location filter.")

cols_to_drop = [
    'TO_CHECK_CITIZENS', 'KY_CD', 'LAW_CAT_CD', 
    'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC',
    'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX',
    'SAME_AGE_GROUP', 'SAME_SEX'
]

actual_cols_to_drop = [col for col in cols_to_drop if col in df_cleaned.columns]
if actual_cols_to_drop:
    df_cleaned = df_cleaned.drop(columns=actual_cols_to_drop)
    print(f"Dropped specified columns: {actual_cols_to_drop}. Columns remaining: {df_cleaned.shape[1]}") 
else:
    print("No specified columns from 'cols_to_drop' found to remove.") 

final_rows = len(df_cleaned)
print(f"Data cleaning completed: {initial_rows} -> {final_rows} rows ({ (f'{final_rows/initial_rows*100:.1f}% retained' if initial_rows > 0 else f'{0.0:.1f}%') })") 

# Clean the data
df = df_cleaned
print(f"Cleaned df shape: {df.shape}") 
if not df.empty:
    print(f"Cleaned df columns: {df.columns.tolist()}") 
else:
    print("Cleaned df is empty after cleaning steps.")

=== Data Cleaning ===
Removed 225707 rows where TO_CHECK_CITIZENS = 0
Removed 577470 rows where VIC_SEX is not 'M' or 'F'. Rows remaining: 1690658
Removed 21713 rows with UNKNOWN LOC_OF_OCCUR_DESC. Rows remaining: 1668945
Dropped specified columns: ['TO_CHECK_CITIZENS', 'KY_CD', 'LAW_CAT_CD', 'OFNS_DESC', 'PD_CD', 'PREM_TYP_DESC', 'SUSP_AGE_GROUP', 'SUSP_RACE', 'SUSP_SEX', 'SAME_AGE_GROUP', 'SAME_SEX']. Columns remaining: 33
Data cleaning completed: 2493835 -> 1668945 rows (66.9% retained)
Cleaned df shape: (1668945, 33)
Cleaned df columns: ['BORO_NM', 'LOC_OF_OCCUR_DESC', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'HOUR', 'DAY', 'WEEKDAY', 'IS_WEEKEND', 'MONTH', 'YEAR', 'SEASON',

### Reduce the number of rows

In [6]:
print("=== Applying Temporal Filter (YEAR >= 2023) ===") 
if 'YEAR' in df.columns:
    initial_rows_temporal_filter = len(df)
    df = df[df['YEAR'] >= 2023].copy()
    rows_after_temporal_filter = len(df)
    print(f"Filtered data to include only rows from 2023 onwards. Rows before: {initial_rows_temporal_filter}, Rows after: {rows_after_temporal_filter}") 
    if df.empty:
        print("DataFrame is empty after temporal filter.")
else:
    print("'YEAR' column not found. Skipping temporal filter.") 

=== Applying Temporal Filter (YEAR >= 2023) ===
Filtered data to include only rows from 2023 onwards. Rows before: 1668945, Rows after: 714420


## STKDE: parameter optimization only

The `stkde_intensity` column and the `RISK_LEVEL` label are **not** computed here to avoid leakage. Only parameter optimization is performed on X_train. Actual feature/label engineering will take place in `Modeling.ipynb`.

**Note:**
- `stkde_intensity` and `RISK_LEVEL` are not computed here to prevent data leakage.
- Only parameter optimization is performed here; feature and label engineering are postponed to the modeling phase.

In [7]:
def calculate_stkde_intensity(
    df: pd.DataFrame,
    year_col: str,
    month_col: str,
    day_col: str,
    hour_col: str,
    lat_col: str,
    lon_col: str,
    hs: float = 200.0,
    ht: float = 60.0
) -> pd.DataFrame:
    """
    Calculate STKDE intensity for each event in the DataFrame.
    Spatial coords are projected to EPSG:3857 (meters) and time
    is normalized in days relative to the earliest event.
    """

    print("=== Calculating STKDE Intensity (Projected + Relative Time) ===")
    start_time = time.time()

    # Quick checks
    if hs <= 0 or ht <= 0 or df.empty:
        print("Invalid parameters or empty DataFrame, returning NaN intensities.")
        df_out = df.copy()
        df_out['stkde_intensity'] = np.nan
        return df_out

    df_copy = df.copy()
    required = [year_col, month_col, day_col, hour_col, lat_col, lon_col]
    missing = [c for c in required if c not in df_copy.columns]
    if missing:
        print(f"Missing columns {missing}, returning NaN intensities.")
        df_copy['stkde_intensity'] = np.nan
        return df_copy

    # 1) build temporary datetime
    rename_map = {year_col: 'year', month_col: 'month', day_col: 'day', hour_col: 'hour'}
    temp_dt = '__temp_stkde_datetime__'
    try:
        for c in [year_col, month_col, day_col, hour_col]:
            df_copy[c] = pd.to_numeric(df_copy[c], errors='coerce')
        df_copy[temp_dt] = pd.to_datetime(
            df_copy[[year_col, month_col, day_col, hour_col]].rename(columns=rename_map), errors='coerce'
        )
        df_copy.dropna(subset=[temp_dt, lat_col, lon_col], inplace=True)
        df_copy.reset_index(drop=True, inplace=True)
    except Exception as e:
        print(f"Datetime construction error: {e}")
        df_copy['stkde_intensity'] = np.nan
        return df_copy

    if df_copy.empty:
        df_copy['stkde_intensity'] = np.nan
        return df_copy

    # 2) project lat/lon to meters
    transformer = Transformer.from_crs("epsg:4326", "epsg:3857", always_xy=True)
    xs, ys = transformer.transform(df_copy[lon_col].values, df_copy[lat_col].values)
    df_copy['x_m'], df_copy['y_m'] = xs, ys

    # 3) compute relative time in days
    t0 = df_copy[temp_dt].min()
    df_copy['t_days'] = (df_copy[temp_dt] - t0) / np.timedelta64(1, 'D')

    coords = df_copy[['x_m', 'y_m']].values
    times = df_copy['t_days'].values
    tree = KDTree(coords, metric='euclidean')

    # kernel functions
    def spatial_kernel(d, h):
        if h <= 0:
            return 0.0
        return np.exp(-0.5 * (d/h)**2) / (2 * np.pi * h**2)

    def temporal_kernel(dt, h):
        if h <= 0:
            return 0.0
        return np.exp(-dt/h) / (2 * h)

    def process_event(i):
        neigh_idx = tree.query_radius(coords[i:i+1], r=5*hs, return_distance=False)[0]
        s = 0.0
        for j in neigh_idx:
            if i == j:
                continue
            dt = abs(times[i] - times[j])
            if dt>5*ht:
                continue
            dist = np.linalg.norm(coords[i] - coords[j])
            s += spatial_kernel(dist, hs) * temporal_kernel(dt, ht)
        return max(s, 1e-12)

    # parallel computation
    n = len(df_copy)
    try:
        intensities = Parallel(n_jobs=-1)(
            delayed(process_event)(i) for i in range(n)
        )
        df_copy['stkde_intensity'] = np.array(intensities)
    except Exception as e:
        print(f"Parallel STKDE error: {e}")
        df_copy['stkde_intensity'] = np.nan

    # clean up
    df_copy.drop(columns=[temp_dt, 'x_m', 'y_m', 't_days'], errors='ignore', inplace=True)
    end_time = time.time()
    print(f"STKDE completed on {n} events in {end_time-start_time:.2f} seconds.")
    return df_copy

## Data splitting

The creation of the 'RISK_LEVEL' column using qcut on the entire dataset has been disabled to prevent data leakage. This operation is performed only in the modeling pipeline, after the train/test split and within each cross-validation fold.

In [8]:
print("=== Starting Data Splitting ===") 

# Initialize variables at the beginning of the cell
perform_temporal_split_successfully = False
unique_year_months = [] 
X_train, X_test = pd.DataFrame(), pd.DataFrame()
y_train, y_test = pd.Series(name='DUMMY_TARGET', dtype='object'), pd.Series(name='DUMMY_TARGET', dtype='object')

# Ensure YEAR and MONTH are numeric and sortable for temporal split
if not df.empty and 'YEAR' in df.columns and 'MONTH' in df.columns:
    df['YEAR'] = pd.to_numeric(df['YEAR'], errors='coerce')
    df['MONTH'] = pd.to_numeric(df['MONTH'], errors='coerce')
    df.dropna(subset=['YEAR', 'MONTH'], inplace=True)
    print(f"Ensured YEAR and MONTH are numeric. Rows after NaN drop (if any): {len(df)}") 
    
    if df.empty: # Check if df became empty after dropna
           print("DataFrame became empty after YEAR/MONTH processing. Cannot perform split.")
    else:
        df['YearMonth'] = df['YEAR'] * 100 + df['MONTH']
        df.sort_values('YearMonth', inplace=True)
        print("Created 'YearMonth' for sorting and df sorted.") 
        print(f"Distribution of data across YearMonth:\n{df['YearMonth'].value_counts().sort_index()}") 
        unique_year_months = df['YearMonth'].unique()
else:
    print("DataFrame is empty or YEAR/MONTH columns missing. Cannot perform temporal split based on YearMonth.")
    if df.empty:
        print("DataFrame is empty. Cannot perform split.")

# Attempt temporal split
if not df.empty and len(unique_year_months) >= 2: # Check df.empty again as it might have become empty
    total_rows = len(df)
    target_test_rows = total_rows * 0.18
    print(f"Attempting temporal split. Total rows: {total_rows}, target test rows (~20%): {target_test_rows:.0f}") 

    accumulated_rows_for_test = 0
    num_months_for_test_set = 0
    # This will be the OLDEST YearMonth included in the test set
    determined_test_set_start_ym = None

    # Iterate unique_year_months from newest to oldest
    for i in range(len(unique_year_months) - 1, -1, -1):
        ym = unique_year_months[i]
        rows_this_month = df[df['YearMonth'] == ym].shape[0]

        accumulated_rows_for_test += rows_this_month
        num_months_for_test_set += 1
        determined_test_set_start_ym = ym # This ym is the oldest month in the current selection

        if accumulated_rows_for_test >= target_test_rows:
            # Found the minimum number of recent months to meet the target
            break

    print(f"Temporal split scan: Test set would start from YearMonth {determined_test_set_start_ym}, include {num_months_for_test_set} month(s), accumulating {accumulated_rows_for_test} rows.") 

    if determined_test_set_start_ym is not None:
        if determined_test_set_start_ym > unique_year_months[0]: # Ensures training set is not empty
            X_train = df[df['YearMonth'] < determined_test_set_start_ym].copy()
            X_test = df[df['YearMonth'] >= determined_test_set_start_ym].copy()
            # Define y_train and y_test for successful temporal split
            y_train = pd.Series([0] * len(X_train), name='DUMMY_TARGET', dtype='object')
            y_test = pd.Series([0] * len(X_test), name='DUMMY_TARGET', dtype='object')

            print(f"Temporal split successful: Test set starts from YearMonth {determined_test_set_start_ym}.") 
            print(f"  Test set rows: {len(X_test)} (~{len(X_test)/total_rows*100:.2f}% of data).") 
            print(f"  Train set rows: {len(X_train)}.") 
            perform_temporal_split_successfully = True
        else:
            print("Temporal split would result in an empty training set. Falling back to random split.")
            # perform_temporal_split_successfully remains False
    else:
        print("Could not determine a temporal cutoff for splitting. Falling back to random split.")
        # perform_temporal_split_successfully remains False

elif not df.empty: # If df not empty but unique_year_months < 2
    print(f"Not enough unique YearMonth values ({len(unique_year_months)}) for temporal split. Falling back to random split.")
    # perform_temporal_split_successfully remains False

# Fallback to random split if temporal split was not successful and df is not empty
if not perform_temporal_split_successfully and not df.empty:
    print("Performing random split (20% test size, random_state=42).") 
    random_state = 42
    # Stratify=None because y is a dummy target.
    # Ensure df passed to train_test_split does not have 'YearMonth' if it was added
    df_for_split = df.copy()
    if 'YearMonth' in df_for_split.columns:
        df_for_split.drop(columns=['YearMonth'], inplace=True)

    X_train, X_test, y_train, y_test = train_test_split(
        df_for_split, pd.Series([0] * len(df_for_split), name='DUMMY_TARGET', dtype='object'), 
        test_size=0.2, random_state=random_state, stratify=None
    )
    print("Random split performed.") 
elif df.empty and not perform_temporal_split_successfully: # df is empty
    print("DataFrame is empty. Split resulted in empty sets.")

# Drop 'YearMonth' from X_train, X_test, and original X (if it was added to X directly)
if 'YearMonth' in X_train.columns: X_train.drop(columns=['YearMonth'], inplace=True)
if 'YearMonth' in X_test.columns: X_test.drop(columns=['YearMonth'], inplace=True)
if 'YearMonth' in df.columns: df.drop(columns=['YearMonth'], inplace=True) # Clean from original df too

print(f"Training set size (X_train): {X_train.shape[0]} rows, {X_train.shape[1]} columns") 
print(f"Test set size (X_test): {X_test.shape[0]} rows, {X_test.shape[1]} columns") 
print(f"y_train size: {y_train.shape[0]}, y_test size: {y_test.shape[0]}") 

if not X_train.empty:
    print(f"X_train columns: {X_train.columns.tolist()}") 
print("X_train will now be used for STKDE parameter LCV and scaler selection.") 

=== Starting Data Splitting ===
Ensured YEAR and MONTH are numeric. Rows after NaN drop (if any): 714420
Created 'YearMonth' for sorting and df sorted.
Distribution of data across YearMonth:
YearMonth
202301    28904
202302    25824
202303    29259
202304    28931
202305    32076
202306    31626
202307    32832
202308    32030
202309    30193
202310    31450
202311    29141
202312    29832
202401    28495
202402    26824
202403    28991
202404    28657
202405    31861
202406    31398
202407    31752
202408    30787
202409    30246
202410    30900
202411    27822
202412    24589
Name: count, dtype: int64
Attempting temporal split. Total rows: 714420, target test rows (~20%): 128596
Temporal split scan: Test set would start from YearMonth 202408, include 5 month(s), accumulating 144344 rows.
Temporal split successful: Test set starts from YearMonth 202408.
  Test set rows: 144344 (~20.20% of data).
  Train set rows: 570076.
Training set size (X_train): 570076 rows, 33 columns
Test set si

In [9]:
def sampling(df_lcv, max_rows=20000):
    if len(df_lcv) <= max_rows:
        return df_lcv

    print(f"Strategic sampling from {len(df_lcv)} to {max_rows} events...")

    # Create temporal strata to ensure coverage of all months/years
    df_temp = df_lcv.copy()

    # Temporal strata: Year-Month
    df_temp['time_stratum'] = df_temp['YEAR'].astype(str) + '-' + df_temp['MONTH'].astype(str).str.zfill(2)

    # Spatial strata: Grid optimized for NYC
    # Use grid that captures neighborhoods/precincts
    lat_bin_size = 0.01  # about 1.1 km - reasonable size for NYC
    lon_bin_size = 0.01  # about 0.85 km in NYC

    df_temp['lat_bin'] = (df_temp['Latitude'] // lat_bin_size) * lat_bin_size
    df_temp['lon_bin'] = (df_temp['Longitude'] // lon_bin_size) * lon_bin_size
    df_temp['spatial_stratum'] = df_temp['lat_bin'].astype(str) + '_' + df_temp['lon_bin'].astype(str)

    # Combined spatio-temporal stratum
    df_temp['combined_stratum'] = df_temp['time_stratum'] + '_' + df_temp['spatial_stratum']

    # Strata analysis
    stratum_counts = df_temp['combined_stratum'].value_counts()
    n_strata = len(stratum_counts)
    target_per_stratum = max(1, max_rows // n_strata)

    print(f"Spatio-temporal strata: {n_strata}")
    print(f"Target per stratum: {target_per_stratum}")
    print(f"Strata distribution: min={stratum_counts.min()}, max={stratum_counts.max()}, median={stratum_counts.median()}")

    # Adaptive sampling strategy
    sampled_dfs = []

    for stratum, group in df_temp.groupby('combined_stratum'):
        n_in_stratum = len(group)

        if n_in_stratum <= target_per_stratum:
            # If the stratum is small, take all
            sampled_dfs.append(group)
        else:
            # Subsampling with density: in very dense areas, sample less densely to avoid overrepresenting hotspots
            if n_in_stratum > target_per_stratum * 3:
                # For very dense strata, use uniform spatial sampling
                sample_size = min(target_per_stratum * 2, n_in_stratum)
                sampled = group.sample(n=sample_size, random_state=42)
            else:
                # For moderately dense strata, sample proportionally
                sample_size = target_per_stratum
                sampled = group.sample(n=sample_size, random_state=42)

            sampled_dfs.append(sampled)

    # Combine all samples
    df_sampled = pd.concat(sampled_dfs, ignore_index=True)

    # If we are still above the target, do a second round of sampling
    if len(df_sampled) > max_rows:
        print(f"Second round: reducing from {len(df_sampled)} to {max_rows}")
        # Final sampling maintaining temporal proportions
        time_proportions = df_sampled['time_stratum'].value_counts(normalize=True)
        final_samples = []

        for time_stratum, proportion in time_proportions.items():
            group = df_sampled[df_sampled['time_stratum'] == time_stratum]
            n_to_sample = max(1, int(max_rows * proportion))
            if len(group) <= n_to_sample:
                final_samples.append(group)
            else:
                final_samples.append(group.sample(n=n_to_sample, random_state=42))

        df_sampled = pd.concat(final_samples, ignore_index=True)

    # Cleanup temporary columns
    columns_to_drop = ['time_stratum', 'lat_bin', 'lon_bin', 'spatial_stratum', 'combined_stratum']
    df_sampled = df_sampled.drop(columns=[col for col in columns_to_drop if col in df_sampled.columns])

    print(f"Sampling completed: {len(df_sampled)} events selected")
    return df_sampled

In [10]:
# Initialize STKDE parameters and related variables at the beginning of the cell
hs_opt = None
ht_opt = None
required_cols_for_stkde = ['YEAR', 'MONTH', 'DAY', 'HOUR', 'Latitude', 'Longitude']
params_file_path = os.path.join(save_dir, "stkde_optimal_params.json")
lcv_needed = True

print(f"Checking for existing STKDE optimal parameters at: {params_file_path}")
if os.path.exists(params_file_path):
    try:
        with open(params_file_path, 'r') as f:
            params = json.load(f)
        hs_opt = params.get('hs_opt')
        ht_opt = params.get('ht_opt')
        if isinstance(hs_opt, (int, float)) and isinstance(ht_opt, (int, float)):
            print(f"Loaded optimal STKDE parameters from {params_file_path}: hs_opt={hs_opt}, ht_opt={ht_opt}")
            lcv_needed = False
        else:
            print(f"Invalid or non-numeric parameters in {params_file_path}; will run LCV.")
    except Exception as e:
        print(f"Error loading parameters from {params_file_path}: {e}; will run LCV.")
else:
    print(f"No STKDE parameter file found at {params_file_path}; will run LCV.")

if lcv_needed:
    print("=== Likelihood Cross-Validation for STKDE Parameter Optimization ===")
    # Check that X_train has all required columns
    if X_train.empty or not all(col in X_train.columns for col in required_cols_for_stkde):
        print(f"X_train is empty or missing required columns {required_cols_for_stkde}. Skipping LCV.")
    else:
        print(f"Performing LCV on X_train (shape: {X_train.shape})")
        # Drop rows with missing STKDE inputs
        df_lcv = X_train[required_cols_for_stkde].dropna().reset_index(drop=True)
        if df_lcv.empty:
            print("No data left for LCV after dropping NaNs. Skipping LCV.")
        else:
            # Sample to limit size
            max_lcv_sample_size = 20000
            df_lcv_sampled = sampling(df_lcv, max_lcv_sample_size)
            print(f"Starting LCV on {len(df_lcv_sampled)} sampled events.")

            # Define focused grid for bandwidths
            # From this grid we obtain hs = 300 m, ht = 45 days
            # hs_values_new = [100, 200, 300, 500, 800, 1000, 1500]
            # ht_values_new = [1, 3, 7, 14, 21, 30, 45]
            hs_values_new = [150, 250, 300, 350, 450]   
            ht_values_new = [45, 60, 75, 90, 120, 150]   
            print(f"[Focused Grid] hs_values={hs_values_new}, ht_values={ht_values_new}")

            def evaluate_lcv_candidate(params_eval):
                hs_candidate, ht_candidate = params_eval
                # Use the unified calculate_stkde_intensity (which handles datetime and projections)
                df_int = calculate_stkde_intensity(
                    df_lcv_sampled.copy(),
                    year_col='YEAR', month_col='MONTH',
                    day_col='DAY', hour_col='HOUR',
                    lat_col='Latitude', lon_col='Longitude',
                    hs=hs_candidate, ht=ht_candidate
                )
                vals = df_int['stkde_intensity']
                valid = vals[(vals > 1e-12) & np.isfinite(vals)]
                if len(valid) == 0:
                    return -np.inf, hs_candidate, ht_candidate
                return np.mean(np.log(valid)), hs_candidate, ht_candidate

            # Prepare all candidate pairs
            param_candidates = [(h, t) for h in hs_values_new for t in ht_values_new]
            results = Parallel(n_jobs=-1)(
                delayed(evaluate_lcv_candidate)(cand) for cand in param_candidates
            )

            # Select the best candidate
            best_score, best_hs, best_ht = max(results, key=lambda x: x[0])
            print(f"[Focused Grid] Best LCV params: hs={best_hs}, ht={best_ht}, Score={best_score}")

            if np.isfinite(best_score):
                hs_opt, ht_opt = best_hs, best_ht
                print("--- LCV Optimization Complete ---")
                print(f"Optimal parameters: hs_opt={hs_opt} m, ht_opt={ht_opt} days")
                try:
                    with open(params_file_path, 'w') as f:
                        json.dump({'hs_opt': hs_opt, 'ht_opt': ht_opt}, f)
                    print(f"Saved optimal STKDE parameters to {params_file_path}")
                except Exception as e:
                    print(f"Error saving STKDE parameters: {e}")
            else:
                print("No valid intensities found for any candidate. Parameters file not updated.")
else:
    print("Skipping STKDE LCV run; existing parameters were loaded.")

# Fallback to defaults if needed
if hs_opt is None:
    print("hs_opt is None after LCV/loading; falling back to default 200.0")
    hs_opt = 200.0
if ht_opt is None:
    print("ht_opt is None after LCV/loading; falling back to default 60.0")
    ht_opt = 60.0

print(f"Using STKDE parameters: hs_opt = {hs_opt} m, ht_opt = {ht_opt} days")

Checking for existing STKDE optimal parameters at: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Classification (Preprocessing)\stkde_optimal_params.json
No STKDE parameter file found at C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Classification (Preprocessing)\stkde_optimal_params.json; will run LCV.
=== Likelihood Cross-Validation for STKDE Parameter Optimization ===
Performing LCV on X_train (shape: (570076, 33))
Strategic sampling from 570076 to 20000 events...
Spatio-temporal strata: 15706
Target per stratum: 1
Strata distribution: min=1, max=354, median=21.0
Second round: reducing from 29222 to 20000
Sampling completed: 19991 events selected
Starting LCV on 19991 sampled events.
[Focused Grid] hs_values=[150, 250, 300, 350, 450], ht_values=[45, 60, 75, 90, 120, 150]
[Focused Grid] Best LCV params: hs=250, ht=75, Score=-17.48776529303316
--- LCV Optimization Complete ---
Optimal parameters: hs_opt=250 m, ht_opt=75 days
Saved optimal STKDE paramete

# Preprocessing Pipeline (Encoding, Scaling, Feature Selection, PCA)

Preprocessing steps are defined for different model types, using scikit-learn's `Pipeline` and `ColumnTransformer` to avoid data leakage. Pipelines are fit only on the training set in the modeling phase.

This includes:
- **Ordinal encoding** for ordinal categorical features (in general and tree pipelines).
- **One-hot encoding** for nominal categorical features.
- **Cyclical encoding** for cyclical features (custom transformer).
- **Scaling** (RobustScaler) for general pipeline.
- **Feature selection** (RandomForest importance) in general and tree pipelines.
- **Dimensionality reduction** (PCA) for general pipeline.

All column lists are dynamically checked against the columns present in `X_train`.

**Best Practice Check:**

- The general pipeline (with scaling and PCA) is used for linear models and distance-based models (e.g., LogisticRegression, KNN, SVC).
- The tree pipeline (OrdinalEncoder for all categoricals, no scaling/PCA) is used for tree-based models (DecisionTree, RandomForest, GradientBoosting).
- All pipelines are fit only on the training set, and then applied to the test set, avoiding data leakage.

This structure is correct and follows best practices for model-specific preprocessing.

In [11]:
print("=== Defining Preprocessing Pipelines ===") 

# Define chosen_scaler_class before it is used
from sklearn.preprocessing import RobustScaler 
chosen_scaler_class = RobustScaler # Default scaler class, can be changed later

# Ensure X_train exists and has columns before proceeding
if 'X_train' not in locals() or X_train.empty:
    print("X_train is not available or is empty. Cannot define pipelines.")
else:
    train_cols = X_train.columns.tolist()
    print(f"X_train columns available for pipeline definition: {train_cols[:15]}...") 

    # --- Define column types ---
    ordinal_cols_def  = ['VIC_AGE_GROUP']
    nominal_cols_def  = ['BORO_NM', 'LOC_OF_OCCUR_DESC', 'VIC_RACE', 'VIC_SEX', 'SEASON', 'TIME_BUCKET']
    cyclical_cols_def = ['HOUR', 'DAY', 'WEEKDAY', 'MONTH']
    numeric_cols_def  = [
        'Latitude','Longitude','BAR_DISTANCE','NIGHTCLUB_DISTANCE','ATM_DISTANCE',
        'ATMS_COUNT','BARS_COUNT','BUS_STOPS_COUNT','METROS_COUNT','NIGHTCLUBS_COUNT',
        'SCHOOLS_COUNT','METRO_DISTANCE','MIN_POI_DISTANCE','AVG_POI_DISTANCE',
        'MAX_POI_DISTANCE','TOTAL_POI_COUNT','POI_DIVERSITY','POI_DENSITY_SCORE',
        'YEAR'
    ]    
    binary_cols_def   = ['IS_WEEKEND','IS_HOLIDAY','IS_PAYDAY']

    # --- Dynamically filter columns based on what's available in X_train ---
    ordinal_cols = [col for col in ordinal_cols_def if col in train_cols]
    nominal_cols = [col for col in nominal_cols_def if col in train_cols]
    cyclical_cols = [col for col in cyclical_cols_def if col in train_cols]
    numeric_cols = [col for col in numeric_cols_def if col in train_cols]
    binary_cols = [col for col in binary_cols_def if col in train_cols] # Binary cols will be passed through

    print(f"Ordinal columns found in X_train: {ordinal_cols}")
    print(f"Nominal columns found in X_train: {nominal_cols}")
    print(f"Cyclical columns found in X_train: {cyclical_cols}")
    print(f"Numeric columns found in X_train: {numeric_cols}")

    # --- Common Transformers ---
    cyclical_transformer = FunctionTransformer(cyclical_transform, validate=False)
    ordinal_categories = [['<18', '18-24', '25-44', '45-64', '65+', 'UNKNOWN']]
    random_state = 42

    # --- Pipeline for General Models (Linear, KNN, SVC) ---
    transformers_general = []
    if ordinal_cols:
        transformers_general.append(('ord', OrdinalEncoder(categories=ordinal_categories, handle_unknown='use_encoded_value', unknown_value=-1), ordinal_cols))
    if nominal_cols:
        transformers_general.append(('nom', OneHotEncoder(handle_unknown='ignore', sparse_output=False, dtype=np.uint8), nominal_cols))
    if cyclical_cols:
        transformers_general.append(('cyc', cyclical_transformer, cyclical_cols))
    if numeric_cols: 
        transformers_general.append(('num', chosen_scaler_class(), numeric_cols))

    if not transformers_general:
        print("Warning: No transformers applicable for the general pipeline. It will be empty.")
        preprocessor_general = ColumnTransformer([], remainder='passthrough')
    else:
        preprocessor_general = ColumnTransformer(transformers_general, remainder='passthrough', n_jobs=-1)

    preprocessing_pipeline_general = Pipeline([
        ('preprocessor', preprocessor_general),
        ('pca', PCA(n_components=0.95, random_state=random_state))
    ])
    print(f"\nGeneral preprocessing pipeline defined using {chosen_scaler_class.__name__} and PCA.")

    # --- Pipeline for Tree-Based Models (and Naive Bayes) ---
    # Trees benefit from OrdinalEncoding for all categoricals, no scaling, and no PCA.
    all_categorical_cols_for_trees = ordinal_cols + nominal_cols
    
    transformers_trees = []
    if all_categorical_cols_for_trees:
        # We can use a single OrdinalEncoder step. It can handle specified categories for some columns
        # and infer them for others, but it's cleaner to separate them.
        # Let's use one encoder for all categoricals for simplicity, as trees can handle it.
        transformers_trees.append(('ord', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), all_categorical_cols_for_trees))
    
    if cyclical_cols:
        transformers_trees.append(('cyc', cyclical_transformer, cyclical_cols))

    if not transformers_trees:
        print("Warning: No transformers applicable for the tree pipeline. It will be empty.")
        preprocessor_trees = ColumnTransformer([], remainder='passthrough')
    else:
        preprocessor_trees = ColumnTransformer(transformers_trees, remainder='passthrough', n_jobs=-1)

    # This is the full pipeline for tree-based models. It does NOT include scaling or PCA.
    preprocessing_pipeline_trees = Pipeline([
        ('preprocessor', preprocessor_trees)
    ])
    print("Tree-optimized preprocessing pipeline defined (uses OrdinalEncoder, no scaling/PCA).")

=== Defining Preprocessing Pipelines ===
X_train columns available for pipeline definition: ['BORO_NM', 'LOC_OF_OCCUR_DESC', 'VIC_AGE_GROUP', 'VIC_RACE', 'VIC_SEX', 'Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT']...
Ordinal columns found in X_train: ['VIC_AGE_GROUP']
Nominal columns found in X_train: ['BORO_NM', 'LOC_OF_OCCUR_DESC', 'VIC_RACE', 'VIC_SEX', 'SEASON', 'TIME_BUCKET']
Cyclical columns found in X_train: ['HOUR', 'DAY', 'WEEKDAY', 'MONTH']
Numeric columns found in X_train: ['Latitude', 'Longitude', 'BAR_DISTANCE', 'NIGHTCLUB_DISTANCE', 'ATM_DISTANCE', 'ATMS_COUNT', 'BARS_COUNT', 'BUS_STOPS_COUNT', 'METROS_COUNT', 'NIGHTCLUBS_COUNT', 'SCHOOLS_COUNT', 'METRO_DISTANCE', 'MIN_POI_DISTANCE', 'AVG_POI_DISTANCE', 'MAX_POI_DISTANCE', 'TOTAL_POI_COUNT', 'POI_DIVERSITY', 'POI_DENSITY_SCORE', 'YEAR']

General preprocessing pipeline defined using RobustScaler and PCA.
Tree-op

## Saving artifacts

Save the unprocessed train/test sets, pipeline definitions, and scoring metrics for use in the modeling phase.

In [12]:
print("\n=== Saving Unprocessed Train/Test Sets, Pipeline Definitions, and Other Artifacts ===")

os.makedirs(save_dir, exist_ok=True)
print(f"Artifacts will be saved in: {save_dir}")

# Save unprocessed data as pandas DataFrames for easier column handling in the next notebook
if 'X_train' in locals() and not X_train.empty:
    try:
        X_train.to_pickle(os.path.join(save_dir, 'X_train.pkl'))
        print(f"X_train saved as pickle ({X_train.shape}).")
    except Exception as e:
        print(f"Error saving X_train: {e}")
else:
    print("X_train is not available or empty. Not saving X_train.")

if 'X_test' in locals() and not X_test.empty:
    try:
        X_test.to_pickle(os.path.join(save_dir, 'X_test.pkl'))
        print(f"X_test saved as pickle ({X_test.shape}).")
    except Exception as e:
        print(f"Error saving X_test: {e}")
else:
    print("X_test is not available or empty. Not saving X_test.")

# y can be saved as numpy array as it's a single series
if 'y_train' in locals() and not y_train.empty:
    try:
        y_train.to_pickle(os.path.join(save_dir, 'y_train.pkl'))
        print(f"y_train saved as pickle ({y_train.shape}).")
    except Exception as e:
        print(f"Error saving y_train: {e}")
else:
    print("y_train is not available or empty. Not saving y_train.")

if 'y_test' in locals() and not y_test.empty:
    try:
        y_test.to_pickle(os.path.join(save_dir, 'y_test.pkl'))
        print(f"y_test saved as pickle ({y_test.shape}).")
    except Exception as e:
        print(f"Error saving y_test: {e}")
else:
    print("y_test is not available or empty. Not saving y_test.")

# Save unfitted pipelines
if 'preprocessing_pipeline_general' in locals():
    try:
        joblib.dump(preprocessing_pipeline_general, os.path.join(save_dir, 'preprocessing_pipeline_general.joblib'))
        print("Unfitted general preprocessing pipeline saved.")
    except Exception as e:
        print(f"Error saving general preprocessing pipeline: {e}")

if 'preprocessing_pipeline_trees' in locals():
    try:
        joblib.dump(preprocessing_pipeline_trees, os.path.join(save_dir, 'preprocessing_pipeline_trees.joblib'))
        print("Unfitted tree-optimized preprocessing pipeline saved.")
    except Exception as e:
        print(f"Error saving tree-optimized preprocessing pipeline: {e}")

# Save parameters
if 'hs_opt' in locals() and 'ht_opt' in locals() and hs_opt is not None and ht_opt is not None: 
    try:
        with open(os.path.join(save_dir, "stkde_optimal_params.json"), 'w') as f:
            json.dump({'hs_opt': hs_opt, 'ht_opt': ht_opt}, f)
        print("STKDE optimal parameters saved.")
    except Exception as e:
        print(f"Error saving STKDE optimal parameters: {e}")

if 'chosen_scaler_class' in locals():
    try:
        with open(os.path.join(save_dir, "scaler_info.json"), 'w') as f:
            json.dump({'chosen_scaler_class_name': chosen_scaler_class.__name__}, f)
        print("Chosen scaler class name saved.")
    except Exception as e:
        print(f"Error saving chosen_scaler_info: {e}")

print("\nArtifact saving process completed.")


=== Saving Unprocessed Train/Test Sets, Pipeline Definitions, and Other Artifacts ===
Artifacts will be saved in: C:\Users\ferdi\Documents\GitHub\crime-analyzer\JupyterOutputs\Classification (Preprocessing)
X_train saved as pickle ((570076, 33)).
X_test saved as pickle ((144344, 33)).
y_train saved as pickle ((570076,)).
y_test saved as pickle ((144344,)).
Unfitted general preprocessing pipeline saved.
Unfitted tree-optimized preprocessing pipeline saved.
STKDE optimal parameters saved.
Chosen scaler class name saved.

Artifact saving process completed.


# Conclusions and Next Steps

- Data loaded, cleaned, and split into train/test sets.
- STKDE parameters optimized only on training data.
- Preprocessing pipelines defined for different model types.
- All feature and label engineering, as well as pipeline fitting, are postponed to the modeling phase to prevent data leakage.

Proceed with `Modeling.ipynb` for feature engineering, target creation, pipeline fitting, and model training.

In [13]:
print("\n=== Final Data Scan (Unprocessed Training Data) ===") 
if 'X_train' in locals() and not X_train.empty:
    print(f"X_train shape: {X_train.shape}") 
    print(f"Sample of X_train (first 5 rows):\n{X_train.head()}") 

    if 'y_train' in locals() and not y_train.empty:
        print(f"y_train shape: {y_train.shape} (Note: y_train is currently a dummy target)") 
    else:
       print("y_train is not defined or is empty.")
else:
    print("X_train is not defined or is empty. Cannot perform final data scan.")



=== Final Data Scan (Unprocessed Training Data) ===
X_train shape: (570076, 33)
Sample of X_train (first 5 rows):
               BORO_NM LOC_OF_OCCUR_DESC VIC_AGE_GROUP  \
2272796         QUEENS            INSIDE         45-64   
827898       MANHATTAN             FRONT         25-44   
1927854          BRONX            INSIDE         25-44   
1927904          BRONX            INSIDE         25-44   
382      STATEN ISLAND            INSIDE         45-64   

                         VIC_RACE VIC_SEX   Latitude  Longitude  BAR_DISTANCE  \
2272796  ASIAN / PACIFIC ISLANDER       F  40.753403 -73.815551    988.943219   
827898             WHITE HISPANIC       M  40.771866 -73.951560    146.973480   
1927854  ASIAN / PACIFIC ISLANDER       F  40.828117 -73.871285    806.327770   
1927904            WHITE HISPANIC       M  40.828117 -73.871285    806.327770   
382                         WHITE       M  40.511804 -74.250034   3126.908322   

         NIGHTCLUB_DISTANCE  ATM_DISTANCE  ...  H